In [521]:
import os
os.chdir('/home/ubuntu/code/pump_post_midterm/pump')

In [522]:
# # Get test_q_keys
# import random
# import pandas as pd
# from src.utils import get_file_from_s3

# survey_name = "American_Trends_Panel_W34"
# random.seed(42)
# survey_df = pd.read_csv(get_file_from_s3(f"human_resp/{survey_name}/info.csv"))
# q_keys = list(survey_df['key'])
# test_q_keys = random.choices(q_keys, k=5)

# print(test_q_keys)

In [523]:
import random
random.seed(42)

from src.utils import TEST_KEY_MAPPING

test_q_keys_mapping = TEST_KEY_MAPPING

# test_q_keys_mapping = {
#     "American_Trends_Panel_W26": ['GUNRESPNOKIDSB_W26', 'WORLDDANGER_W26', 'GUNIDENTITY_W26', 'REASONGUNC_W26', 'GUNRESPKIDSC_W26'],
#     "American_Trends_Panel_W27": ['CAREGIV3A_W27', 'PREDICTA_W27', 'WORK3C_W27', 'CARS10D_W27', 'CAREGIV7_W27'],
#     "American_Trends_Panel_W29": ['TRAITSD_W29', 'NOWSMK_NHIS_W29', 'HELPHURTE_W29', 'HELPHURTA_W29', 'GIRLSF2A_W29'],
#     "American_Trends_Panel_W32": ['NEIGHINTERB_W32', 'SATLIFEC_W32', 'SUCCESSIMPA_W32', 'COMMIMPE_W32', 'GROWUPUSR_W32'],
#     "American_Trends_Panel_W34": ['MED4A_W34', 'SCI2A_W34', 'FUD22_W34', 'EAT3H_W34', 'MED6D_W34']
# }


# Hyper parameters

In [524]:

survey_names = [
    # "American_Trends_Panel_W26",
    # "American_Trends_Panel_W27",
    # "American_Trends_Panel_W29",
    # "American_Trends_Panel_W32",
    "American_Trends_Panel_W34",
    "American_Trends_Panel_W41",
    "American_Trends_Panel_W82",
]

def skip_exp(filename):

    # check known test
    if 'v17' not in filename or all([_ not in filename for _ in ['0904', '0905', '0909', '0911', '0912']]): return True
    if 'bdemo' not in filename: return True
    # if 'llama' not in filename: return True
    # if '0911' not in filename: return True
    if 'sonnet' not in filename: return True
    # if 'haiku' not in filename: return True

    return False

def get_exp(filename):
    if filename[-5:] == '.json': filename = filename[:-5]
    

    segs = ['history', 'demo', 'persona', 'withname', 'all' 'allq', 'ragupper', 'bdemo', 'rag', 'mostcommon', 'mostcommon40', 'bn', 'date0911', 'date0912']
    segs += [f"bn{bn}{config}" for bn in ['bds', 'bdeu', 'bic', 'none'] for config in [1, 2, 3, 4]]
    for model in ['haiku', 'sonnet', 'llama70b3', 'mistrallarge', 'sonnet35']: segs.append(model + "pred")
    for level in ['13', '16', '17']: segs.append('v'+level)
    for train in ['200', '500', '2000', 'All']: segs.append('train'+train)
    for top in range(10, 110, 10): segs.append('top'+str(top))
    for skew in ['5', '3', '10']: segs.append('skew'+skew)
    for usedemo in ['True', 'False']: segs.append('usedemo'+usedemo)
    for run in ['run1', 'run2', 'run3']: segs.append(run)
    for setting in ['queryponly', 'querydp', 'querydonly']: segs.append(setting)

    exp_elements = [_ for _ in filename.split('_') if _ in segs]
    final = '_'.join(exp_elements)
    return final

# Main

In [525]:
import json
import os
import pandas as pd
import numpy as np
import pandas as pd
import random

from collections import Counter
from src.utils import get_file_from_s3

def get_full_results(survey_name, sort_by):

    with open(f'experiment/data/human_resp/{survey_name}/user_test_q_key_mapping.json', 'r') as f:
        user_test_q_key_mapping = json.load(f)
    
    res = {}
    for filename in os.listdir(f'opinions_qa/output/{survey_name}/'):
        if skip_exp(filename): continue
        exp = get_exp(filename)
        # exp = filename
        if exp not in res:
            res[exp] = []
            
        with open(f"opinions_qa/output/{survey_name}/{filename}", 'r') as f:
            data = json.load(f)
        res[exp] += [f"={sum([_['is_correct'] for _ in data])}/{len(data)}", sum([_['is_correct'] for _ in data])/len(data)]

    for key in res:
        print(key, len(res[key]))

    df = pd.DataFrame(res)
    df = df[sorted(df.columns)]
    df = df.transpose()

    acc_df = df
    acc_df = acc_df.rename(columns={0: "acc_cnt", 1: "acc"})
    # acc_df.sort_values(by='acc', ascending=False)


    survey_df = pd.read_csv(get_file_from_s3(f"human_resp/{survey_name}/info.csv"))

    wd_question_mappings = {row['key']: dict(zip(eval(row['references'].lower()), eval(row['option_ordinal']))) for _, row in survey_df.iterrows()}
    wd_question_mappings

    records = {}
    for filename in os.listdir(f'opinions_qa/output/{survey_name}/'):
        if skip_exp(filename): continue
        # print(filename)
        with open(f"opinions_qa/output/{survey_name}/{filename}", 'r') as f:
            data = json.load(f)
        
        exp = get_exp(filename)

        res = []
        pred_missing_cnt = 0
        gold_missing_cnt = 0
        for entry in data:
            q_idx = entry['q_idx']
            q_key = user_test_q_key_mapping[str(entry["user_idx"])][q_idx]
            pos_mapping = wd_question_mappings[q_key]
            prediction = entry['prediction'].lower()
            gold_answer = entry['gold_answer'].lower()
            if gold_answer not in pos_mapping:
                gold_missing_cnt += 1
                continue
            if prediction not in pos_mapping:
                pred_missing_cnt += 1
                if prediction not in entry['references']:
                    refs = '/'.join(entry['references'].split('\n'))
                    print(f"Prediction: |{prediction}|, Gold: |{gold_answer}|, References: |{refs}|")
                continue
                prediction = random.choice(list(pos_mapping.keys()))
            pred_pos = pos_mapping[prediction]
            gold_pos = pos_mapping[gold_answer]
            res.append(abs(pred_pos-gold_pos))
        records[exp] = [Counter(res), np.mean(res), pred_missing_cnt, gold_missing_cnt]

    q_wd_df = pd.DataFrame(records)
    q_wd_df = q_wd_df[sorted(q_wd_df.columns)].transpose()
    q_wd_df = q_wd_df.rename(columns={0: "q_1wd_cnt", 1: "q_1wd", 2: "q_1wd_pmc", 3: "q_1wd_gmc"})
    q_wd_df

    # print(acc_df)
    # print(q_wd_df)

    merged_df = pd.merge(acc_df, q_wd_df, left_index=True, right_index=True)
    # merged_df = pd.merge(merged_df, s_wd_df, left_index=True, right_index=True)
    # merged_df = merged_df[['wd_avg', 'wd_std', 'acc_avg', 'acc_std']]

    if sort_by == 'q_1wd':
        merged_df = merged_df.sort_values(by='q_1wd', ascending=True)
    elif sort_by == 'acc':
        merged_df = merged_df.sort_values(by='acc', ascending=False)

    # merged_df.sort_values(by='acc_avg', ascending=False)
    # merged_df.sort_values(by='wd_avg', ascending=True)
    # merged_df = merged_df.sort_values(by='s_1wd', ascending=True)

    merged_df['acc_cnt'] = merged_df['acc_cnt'].apply(lambda x: '\''+str(x))
    merged_df
    print('\n\n\n')
    print(merged_df.to_csv())

    return merged_df

# Results

In [526]:
from IPython.display import display

for survey_name in survey_names:
    print(survey_name)
    sort_by = 'acc'
    # sort_by = 'q_1wd'
    merged = get_full_results(survey_name, sort_by=sort_by)
    display(merged)
    print('\n\n\n')

American_Trends_Panel_W34
date0912_v17_bdemo_mostcommon40_bnbdeu1_sonnetpred_history_demo_rag_run2 2
date0911_v17_bdemo_mostcommon40_bn_sonnetpred_history_demo_persona_rag_run2 2
date0911_v17_bdemo_mostcommon40_bnbds2_sonnetpred_history_demo_persona_rag_run1 2
date0912_v17_bdemo_mostcommon40_bnbdeu1_sonnetpred_history_demo_rag_run1 2
date0912_v17_bdemo_mostcommon40_bnnone2_sonnetpred_history_demo_persona_rag_run2 2
date0912_v17_bdemo_mostcommon40_bnbdeu4_sonnetpred_history_demo_persona_rag_run2 2
date0912_v17_bdemo_mostcommon40_bnbdeu4_sonnetpred_history_demo_persona_rag_run1 2
date0911_v17_bdemo_mostcommon40_bnbic1_sonnetpred_history_demo_persona_rag_run1 2
date0911_v17_bdemo_mostcommon40_bnbic2_sonnetpred_history_demo_persona_rag_run2 2
date0912_v17_bdemo_mostcommon40_bnnone1_sonnetpred_history_demo_persona_rag_run2 2
date0911_v17_bdemo_sonnetpred_history_run2 2
date0912_v17_bdemo_mostcommon40_bnnone1_sonnetpred_history_demo_persona_rag_run1 2
date0911_v17_bdemo_mostcommon40_bnbds2_s

,acc_cnt,acc,q_1wd_cnt,q_1wd,q_1wd_pmc,q_1wd_gmc
date0912_v17_bdemo_mostcommon40_bnbdeu1_sonnetpred_history_demo_rag_run2,'=651/1000,0.651,"{0.0: 651, 1.0: 298, 2.0: 22, 3.0: 1, 0.5: 13}",0.356853,2,13
date0912_v17_bdemo_mostcommon40_bnbdeu1_sonnetpred_history_demo_rag_run1,'=651/1000,0.651,"{0.0: 651, 1.0: 298, 2.0: 22, 3.0: 1, 0.5: 13}",0.356853,2,13
date0912_v17_bdemo_mostcommon40_bnnone1_sonnetpred_history_demo_rag_run2,'=645/1000,0.645,"{0.0: 645, 1.0: 304, 2.0: 23, 3.0: 2, 0.5: 12}",0.36714,1,13
date0912_v17_bdemo_mostcommon40_bnnone2_sonnetpred_history_demo_rag_run2,'=641/1000,0.641,"{0.0: 641, 1.0: 312, 2.0: 20, 3.0: 1, 0.5: 11}",0.36599,2,13
date0911_v17_bdemo_sonnetpred_history_run2,'=635/1000,0.635,"{1.0: 326, 0.0: 635, 2.0: 13, 0.5: 9, 3.0: 2}",0.36802,2,13
date0911_v17_bdemo_sonnetpred_history_run3,'=635/1000,0.635,"{1.0: 326, 0.0: 635, 2.0: 13, 0.5: 9, 3.0: 2}",0.36802,2,13
date0911_v17_bdemo_sonnetpred_history_run1,'=634/1000,0.634,"{1.0: 327, 0.0: 634, 2.0: 13, 0.5: 9, 3.0: 2}",0.369036,2,13
date0911_v17_bdemo_mostcommon40_bnbdeu1_sonnetpred_history_demo_persona_rag_run2,'=628/1000,0.628,"{0.0: 628, 1.0: 324, 2.0: 21, 3.0: 1, 0.5: 12}",0.380325,1,13
date0911_v17_bdemo_mostcommon40_bnbdeu2_sonnetpred_history_demo_persona_rag_run1,'=628/1000,0.628,"{0.0: 628, 1.0: 324, 2.0: 21, 3.0: 1, 0.5: 12}",0.380325,1,13
date0912_v17_bdemo_mostcommon40_bnnone1_sonnetpred_history_demo_persona_rag_run1,'=627/1000,0.627,"{0.0: 627, 1.0: 319, 2.0: 21, 3.0: 2, 0.5: 13}",0.380346,5,13






American_Trends_Panel_W41
date0911_v17_bdemo_sonnetpred_history_run3 2
date0911_v17_bdemo_sonnetpred_history_run1 2
date0911_v17_bdemo_mostcommon40_bn_sonnetpred_history_demo_persona_rag_run3 2
date0911_v17_bdemo_mostcommon40_bn_sonnetpred_history_demo_persona_rag_run1 2
date0911_v17_bdemo_sonnetpred_history_run2 2
date0911_v17_bdemo_mostcommon40_bn_sonnetpred_history_demo_persona_rag_run2 2
Prediction: |a top priority|, Gold: |very worried|, References: |Very worried/Fairly worried/Not too worried/Not at all worried/Refused|
Prediction: |a top priority|, Gold: |very worried|, References: |Very worried/Fairly worried/Not too worried/Not at all worried/Refused|
Prediction: |a top priority|, Gold: |very worried|, References: |Very worried/Fairly worried/Not too worried/Not at all worried/Refused|




,acc_cnt,acc,q_1wd_cnt,q_1wd,q_1wd_pmc,q_1wd_gmc
date0911_v17_bdemo_mostcommon40_bn_sonnetpred_history_demo_persona_rag_run2,'=546/1000,0.546,"Counter({0.0: 557, 1.0: 296, 0.5: 79, 2.0: 

,acc_cnt,acc,q_1wd_cnt,q_1wd,q_1wd_pmc,q_1wd_gmc
date0911_v17_bdemo_mostcommon40_bn_sonnetpred_history_demo_persona_rag_run2,'=546/1000,0.546,"{1.0: 296, 0.5: 79, 0.0: 557, 2.0: 40, 3.0: 2,...",0.442523,1,16
date0911_v17_bdemo_mostcommon40_bn_sonnetpred_history_demo_persona_rag_run3,'=546/1000,0.546,"{1.0: 296, 0.5: 79, 0.0: 557, 2.0: 40, 3.0: 2,...",0.442523,1,16
date0911_v17_bdemo_mostcommon40_bn_sonnetpred_history_demo_persona_rag_run1,'=545/1000,0.545,"{1.0: 297, 0.5: 79, 0.0: 556, 2.0: 40, 3.0: 2,...",0.44354,1,16
date0911_v17_bdemo_sonnetpred_history_run1,'=513/1000,0.513,"{0.0: 529, 1.0: 331, 0.5: 70, 2.0: 46, 3.0: 2,...",0.480691,0,16
date0911_v17_bdemo_sonnetpred_history_run2,'=513/1000,0.513,"{0.0: 529, 1.0: 331, 0.5: 70, 2.0: 46, 3.0: 2,...",0.480691,0,16
date0911_v17_bdemo_sonnetpred_history_run3,'=513/1000,0.513,"{0.0: 529, 1.0: 331, 0.5: 70, 2.0: 46, 3.0: 2,...",0.480691,0,16






American_Trends_Panel_W82
date0911_v17_bdemo_mostcommon40_bn_sonnetpred_history_demo_persona_rag_run2 2
date0911_v17_bdemo_sonnetpred_history_run3 2
date0911_v17_bdemo_sonnetpred_history_run1 2
date0911_v17_bdemo_mostcommon40_bn_sonnetpred_history_demo_persona_rag_run3 2
date0911_v17_bdemo_mostcommon40_bn_sonnetpred_history_demo_persona_rag_run1 2
date0911_v17_bdemo_sonnetpred_history_run2 2




,acc_cnt,acc,q_1wd_cnt,q_1wd,q_1wd_pmc,q_1wd_gmc
date0911_v17_bdemo_mostcommon40_bn_sonnetpred_history_demo_persona_rag_run1,'=575/1000,0.575,"Counter({0.0: 575, 1.0: 349, 2.0: 44, 0.5: 10, 3.0: 3})",0.4597349643221203,0,19
date0911_v17_bdemo_mostcommon40_bn_sonnetpred_history_demo_persona_rag_run2,'=575/1000,0.575,"Counter({0.0: 575, 1.0: 349, 2.0: 44, 0.5: 10, 3.0: 3})",0.4597349643221203,0,19
date0911_v17_bdemo_mostcommon40_bn_sonnetpred_history_demo_persona_rag_run3,'=575/1000,0.575,"Counter({0.0: 575, 1.0: 349, 2.0: 44, 0.5: 10, 3.0: 3})",0.4597349643221203,0,19
date0911_v17_bdemo_sonn

,acc_cnt,acc,q_1wd_cnt,q_1wd,q_1wd_pmc,q_1wd_gmc
date0911_v17_bdemo_mostcommon40_bn_sonnetpred_history_demo_persona_rag_run1,'=575/1000,0.575,"{0.0: 575, 1.0: 349, 2.0: 44, 0.5: 10, 3.0: 3}",0.459735,0,19
date0911_v17_bdemo_mostcommon40_bn_sonnetpred_history_demo_persona_rag_run2,'=575/1000,0.575,"{0.0: 575, 1.0: 349, 2.0: 44, 0.5: 10, 3.0: 3}",0.459735,0,19
date0911_v17_bdemo_mostcommon40_bn_sonnetpred_history_demo_persona_rag_run3,'=575/1000,0.575,"{0.0: 575, 1.0: 349, 2.0: 44, 0.5: 10, 3.0: 3}",0.459735,0,19
date0911_v17_bdemo_sonnetpred_history_run1,'=570/1000,0.57,"{0.0: 570, 1.0: 360, 0.5: 12, 2.0: 36, 3.0: 3}",0.455657,0,19
date0911_v17_bdemo_sonnetpred_history_run3,'=570/1000,0.57,"{0.0: 570, 1.0: 360, 0.5: 12, 2.0: 36, 3.0: 3}",0.455657,0,19
date0911_v17_bdemo_sonnetpred_history_run2,'=569/1000,0.569,"{0.0: 569, 1.0: 361, 0.5: 12, 2.0: 36, 3.0: 3}",0.456677,0,19


In [530]:
a = 'aaa'
b = 'bbb'
c = 'ccc'

b = a 
b = 'ccc'
a


'aaa'

In [527]:
def get_q1wd(resp_df, similar_user_mapping, user_test_q_key_mapping, wd_question_mappings):
    res = []
    gold_missing_cnt = 0
    pred_missing_cnt = 0
    for user in similar_user_mapping:
        for key in user_test_q_key_mapping[user]:
            similar_users = [int(_) for _ in similar_user_mapping[user]]
            row = resp_df.iloc[int(user)]
            gold_answer = row[key].lower()
            pos_mapping = wd_question_mappings[key]
            
            similar_df = resp_df[resp_df.index.isin(similar_users)]
            similar_answers = similar_df[key].dropna()
            if len(similar_answers) == 0:
                similar_answers = ["NA"]  # for most common operation in the next line
            prediction = Counter(similar_answers).most_common(1)[0][0].lower()

            if gold_answer not in pos_mapping:
                gold_missing_cnt += 1
                continue
            if prediction not in pos_mapping:
                pred_missing_cnt += 1
                if prediction not in pos_mapping.keys():
                    refs = '/'.join(pos_mapping.keys())
                    print(f"Prediction: |{prediction}|, Gold: |{gold_answer}|, References: |{refs}|")
                continue
            pred_pos = pos_mapping[prediction]
            gold_pos = pos_mapping[gold_answer]

            res.append({
                "user": user,
                "key": key,
                "q1wd": abs(pred_pos-gold_pos)
            })
    
    avg_q1wd = sum([_['q1wd'] for _ in res]) / len(res)
    return avg_q1wd, res


def get_q1wd_for_most_common_and_majority(survey_name, similar_user_mapping_filename):
    with open(f'experiment/data/human_resp/{survey_name}/user_test_q_key_mapping.json', 'r') as f:
        user_test_q_key_mapping = json.load(f)
    with open(similar_user_mapping_filename, 'r') as f:
        rag_similar_user_mapping = json.load(f)
    resp_df = pd.read_csv(get_file_from_s3(f"human_resp/{survey_name}/responses.csv"))
    survey_df = pd.read_csv(get_file_from_s3(f"human_resp/{survey_name}/info.csv"))
    wd_question_mappings = {row['key']: dict(zip(eval(row['references'].lower()), eval(row['option_ordinal']))) for _, row in survey_df.iterrows()}
    
    avg_q1wd, res = get_q1wd(resp_df, rag_similar_user_mapping, user_test_q_key_mapping, wd_question_mappings)
    # print(avg_q1wd, Counter([_['q1wd'] for _ in res]), len(res))
    print(avg_q1wd)

    train_users = [str(_) for _ in range(len(resp_df)) if str(_) not in rag_similar_user_mapping]
    test_to_train_user_mapping = {k: train_users for k in rag_similar_user_mapping}

    avg_q1wd, res = get_q1wd(resp_df, test_to_train_user_mapping, user_test_q_key_mapping, wd_question_mappings)
    # print(avg_q1wd, Counter([_['q1wd'] for _ in res]), len(res))
    print(avg_q1wd)



In [528]:
# survey_to_similar_user = {
#     'American_Trends_Panel_W34': "opinions_qa/similar_users/American_Trends_Panel_W34/date0905_personas_full_personadb_querydp_trainAll_top80_skew10_withname.json",
#     'American_Trends_Panel_W41': "opinions_qa/similar_users/American_Trends_Panel_W41/date0905_personas_full_personadb_bn_queryponly_trainAll_top160_skew15_withname.json",
#     'American_Trends_Panel_W82': "opinions_qa/similar_users/American_Trends_Panel_W82/date0905_personas_full_personadb_bn_hcbdeu_querydp_trainAll_top100_skew2_withname.json",
# }

# for survey_name, similar_user_mapping_filename in list(survey_to_similar_user.items()):
#     get_q1wd_for_most_common_and_majority(survey_name, similar_user_mapping_filename)   